# What happens in this notebook :-)

* We read in MC samples for background and different signal models. We also read in the actual data from the L3 detector.

* We load the pickel file of the trained BDT and select our events according to some cut on the BDT response.

* The selected candidates passing the BDT cut as well as the cut MC samples are stored and used for the 2D analysis in Higgs@L3-2D.ipynb.

* A 1D log-Likelihood ratio analysis is performed.

* Quantile plots are produced


### import packages

In [ ]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import os

import plotting as pl
import helpers

### Read in the data

In [ ]:
from load_data import data, mc_higgs_models, mc_no_higgs_frames

\begin{array}{|l||c|c|c|c|}
\hline
\textrm{sample name} & \textrm{type of data} & \textrm{real data} & \textrm{No. of events} & \sigma   [\textrm{pb}] \\ \hline
\textrm{higgs_qq} & q \bar{q} & - & 200000 & 102 \\ \hline
\textrm{higgs_ww} & W^+ W^- & - & 294500 & 16.5 \\ \hline
\textrm{higgs_zz} & ZZ & - & 196000 & 0.975 \\ \hline
\textrm{higgs_zee} & Z e^+ e^- & - & 29500 & 3.35 \\ \hline
\textrm{higgs_wen} & q \bar{q} e \nu_e & - & 81786 & 2.90 \\ \hline
\textrm{higgs_eeqq} & \textrm{two photon coll} & - & 5940000 & 15600 \\ \hline
\textrm{higgs_data} & \textrm{data} & x & - & - \\ \hline
\textrm{higgs_higgs_85} & \textrm{Higgs} (m_{H} = 85 \ \textrm{GeV}) & - & 3972 & 0.0940\\ \hline
\textrm{higgs_higgs_90} & \textrm{Higgs} (m_{H} = 90 \ \textrm{GeV}) & - & 3973 & 0.0667\\ \hline
\textrm{higgs_higgs_95} & \textrm{Higgs} (m_{H} = 95 \ \textrm{GeV}) & - & 3971 & 0.0333\\ \hline	
\end{array}

### Introduce weights in order to rescale all the MC samples to the same luminosity

per definition the weight of each measured event (data) is $1$. 
Thus, we want to rescale the MC to the same integrated Luminosity of the data taking which is $L = 176.773 \ \mathrm{pb}^{-1}$.

The weight for each MC sample is:
$$ 
        \mathrm{weight}_\mathrm{MC} = L \cdot  \frac{\sigma_\mathrm{MC} }{N_\mathrm{MC}}
$$

In [ ]:
with open("BDT_85higgs1.pkl", "rb") as fid:
    BDT_85 = pickle.load(fid)

with open("BDT_90higgs4.pkl", "rb") as fid:
    BDT_90 = pickle.load(fid)

with open("BDT_95higgs3.pkl", "rb") as fid:
    BDT_95 = pickle.load(fid)
    
for frame in itertools.chain(mc_higgs_models.values(), 
                             mc_no_higgs_frames.values(), 
                             [data]):
    helpers.addComposedVariable(df=frame)
    frame["BDT_selCut85"] = BDT_85.decision_function(frame[helpers.kinematical_vars])
    frame["BDT_selCut90"] = BDT_90.decision_function(frame[helpers.kinematical_vars])
    frame["BDT_selCut95"] = BDT_95.decision_function(frame[helpers.kinematical_vars])


## Cut on the BDT Value.

In [ ]:
cut_85 = -3.15
cut_90 = -2.9
cut_95 = -2.9

data_85 = data[data["BDT_selCut85"] > cut_85][helpers.kinematical_vars + ["weight"]]
data_90 = data[data["BDT_selCut90"] > cut_90][helpers.kinematical_vars + ["weight"]]
data_95 = data[data["BDT_selCut95"] > cut_95][helpers.kinematical_vars + ["weight"]]

df = mc_higgs_models["higgs_85"]
sig_85 = df[df["BDT_selCut85"] > cut_85][helpers.kinematical_vars + ["weight"]]
df = mc_higgs_models["higgs_90"]
sig_90 = df[df["BDT_selCut90"] > cut_90][helpers.kinematical_vars + ["weight"]]
df = mc_higgs_models["higgs_95"]
sig_95 = df[df["BDT_selCut95"] > cut_95][helpers.kinematical_vars + ["weight"]]

df_bkg = pd.concat(mc_no_higgs_frames, ignore_index=True)
bkg_85 = df_bkg[df_bkg["BDT_selCut85"] > cut_85][helpers.kinematical_vars+["weight"]]
bkg_90 = df_bkg[df_bkg["BDT_selCut90"] > cut_90][helpers.kinematical_vars+["weight"]]
bkg_95 = df_bkg[df_bkg["BDT_selCut95"] > cut_95][helpers.kinematical_vars+["weight"]]


In [ ]:
# purity in range 90 +- 5 GeV

sig_in_interval = sig_90[(sig_90['mmis']<95)&(sig_90['mmis']>85)]['weight'].sum()
bkg_in_interval = bkg_90[(bkg_90['mmis']<95)&(bkg_90['mmis']>85)]['weight'].sum()

purity = sig_in_interval*1. / bkg_in_interval

print(purity)
print(len(data_90['weight']))

In [ ]:
data_85.to_csv('data_85')
data_90.to_csv('data_90')
data_95.to_csv('data_95')

sig_85.to_csv('sig_85')
sig_90.to_csv('sig_90')
sig_95.to_csv('sig_95')

bkg_85.to_csv('bkg_85')
bkg_90.to_csv('bkg_90')
bkg_95.to_csv('bkg_95')

### distribution of variable of interest

In [ ]:
path = "plots/VariableDists/"
variable = 'mmis'
x_name = r'missing mass $m_\mathrm{mis}$ '
x_unit = r'GeV/$c^2$'
binning = np.array([50,60,70,80,85,90,95,100,105,110,120,140])
binning = np.linspace(50,130,28)
#binning = np.linspace(75,100,28)
save_path = None#path+variable

In [ ]:
bkg_histos = []
sig_histos = []
data_histos = []

for frame in [bkg_85,bkg_90,bkg_95]:
    bkg_histos.append(np.histogram(
        frame[variable], bins=binning, weights=frame["weight"]
    )[0])

for frame in [sig_85,sig_90,sig_95]:
    sig_histos.append(np.histogram(
        frame[variable], bins=binning, weights=frame["weight"]
    )[0])

for frame in [data_85,data_90,data_95]:
    data_histos.append(np.histogram(
        frame[variable], bins=binning, weights=frame["weight"]
    )[0])    

In [ ]:
pl.BkgSigHistos(background=bkg_histos,
                signals=sig_histos,
                data=data_histos,
                variable_binning=binning,
                x_label=[x_name,x_unit],
                savepath=None)#save_path)

## The Log-likelihood ratio we define to be

$$
    -2 \log (Q(m_H)) = 2 s_{tot} - 2 \sum_{i=1}^{N_{bins}} N_i \log \left( 1 + \frac{s_i(m_H)}{b_i} \right)
$$

In [ ]:
import stats as stat

In [ ]:
"""Nexp = 10000
llr_85 = stat.LogLikRatio(background=bkg,signal=sigModels[0],N_experiments=Nexp)
llr_90 = stat.LogLikRatio(background=bkg,signal=sigModels[1],N_experiments=Nexp)
llr_95 = stat.LogLikRatio(background=bkg,signal=sigModels[2],N_experiments=Nexp)

llr_obs = stat.LogLikRatioObserved(background=bkg,signals=sigModels,data=data_hist)
"""

Nexp = 10000
llr_85 = stat.LogLikRatio(background=bkg_histos[0],signal=sig_histos[0],N_experiments=Nexp)
llr_90 = stat.LogLikRatio(background=bkg_histos[1],signal=sig_histos[1],N_experiments=Nexp)
llr_95 = stat.LogLikRatio(background=bkg_histos[2],signal=sig_histos[2],N_experiments=Nexp)

llr_obs_85 = stat.LogLikRatioObserved(background=bkg_histos[0],signals=sig_histos,data=data_histos[0])[0]
llr_obs_90 = stat.LogLikRatioObserved(background=bkg_histos[1],signals=sig_histos,data=data_histos[1])[1]
llr_obs_95 = stat.LogLikRatioObserved(background=bkg_histos[2],signals=sig_histos,data=data_histos[2])[2]

llr_obs = [llr_obs_85,llr_obs_90,llr_obs_95]

In [ ]:
path = "plots/loglikeliratio/"
save_path = None#path+variable

In [ ]:
print(llr_obs)

In [ ]:
CLlist, Quantiles_b, Quantiles_sPlusb = pl.LogLikRatioPlots([llr_85,llr_90,llr_95],
                                                            Nbins=300,obs=llr_obs,
                                                            savepath=None#save_path
                                                           )
print(CLlist)

$$CL_s = \frac{CL_{s+b}}{CL_b} = \frac{CL_{s+b}}{1-(1-CL_b)}$$

In [ ]:
CLs = [entry[1]/(1-entry[0]) for entry in CLlist]
print(CLs)

print("Confidence level CL >= ")
for i in range(len(CLs)):
    print("    ", 1 - CLs[i])

# Quantile plot

In [ ]:
print(Quantiles_b[0])
print(Quantiles_b[1])
print(Quantiles_b[2])

# print(Quantiles_sPlusb)

In [ ]:
print(Quantiles_b[:][0][0])

In [ ]:
MHiggs = np.array([85.,90.,95])

In [ ]:
Quantiles_b = [[4.17450166378066, [0.36682136305160284, 7.6648752727822966], [-3.9168189752685834, 10.837942190056509]], [2.5979840592736423, [-0.46927828916865977, 5.3244394801112449], [-3.9909498744172254, 7.7100879733441445]], [0.70195337492436138, [-0.90642270726311835, 2.0884844802583942], [-2.7921050105174032, 3.2531706087389818]]]
Quantiles_sPlusb = [[-5.3446990880419776, [-10.738912847408141, -0.42644536626695029], [-16.609086644365437, 4.015848317916948]], [-3.0821314008046912, [-6.9446099136579615, 0.5531424936454421], [-11.261497663317497, 3.8476094604908759]], [-0.68457773040967318, [-2.6257212778773198, 0.9792595959911683], [-4.8441710464117724, 2.4212519455385628]]]

In [ ]:
plt.figure(figsize=(9,7))



y_b = np.array([Quantiles_b[0][0],Quantiles_b[1][0],Quantiles_b[2][0]])
y_b_oneS_lower = np.array([Quantiles_b[0][1][0],Quantiles_b[1][1][0],Quantiles_b[2][1][0]])
y_b_oneS_upper = np.array([Quantiles_b[0][1][1],Quantiles_b[1][1][1],Quantiles_b[2][1][1]])


plt.plot(MHiggs,y_b,'b--',label='expected bkg',linewidth=3)


plt.fill_between(MHiggs, y_b_oneS_lower, y_b_oneS_upper,facecolor='lawngreen',alpha=0.99)




#plt.plot(MHiggs,[Quantiles_b[0][1][0],Quantiles_b[1][1][0],Quantiles_b[2][1][0]],'y-',color='lawngreen')
#plt.plot(MHiggs,[Quantiles_b[0][1][1],Quantiles_b[1][1][1],Quantiles_b[2][1][1]],'y-',color='lawngreen')


y_b_twoS_lower = np.array([Quantiles_b[0][2][0],Quantiles_b[1][2][0],Quantiles_b[2][2][0]])
y_b_twoS_upper = np.array([Quantiles_b[0][2][1],Quantiles_b[1][2][1],Quantiles_b[2][2][1]])

plt.fill_between(MHiggs, y_b_twoS_lower, y_b_oneS_lower,facecolor='yellow',alpha=0.9)
plt.fill_between(MHiggs, y_b_oneS_upper, y_b_twoS_upper,facecolor='yellow',alpha=0.9)


#plt.plot(MHiggs,[Quantiles_b[0][2][0],Quantiles_b[1][2][0],Quantiles_b[2][2][0]],'g-',color='yellow')
#plt.plot(MHiggs,[Quantiles_b[0][2][1],Quantiles_b[1][2][1],Quantiles_b[2][2][1]],'g-',color='yellow')





plt.plot(MHiggs,[Quantiles_sPlusb[0][0],Quantiles_sPlusb[1][0],Quantiles_sPlusb[2][0]],'-.',label='expected sig+bkg',
        color='brown',linewidth=3)


plt.plot(MHiggs,llr_obs,'r-',label='observed',linewidth=3)

plt.hlines(0,85,95,linewidth=1)

plt.xlim(85.,95.)
plt.ylim(-6,13)
plt.xlabel(r'$m_{H}$ [GeV]',fontsize=14)
plt.ylabel(r'$-2 \ln (Q)$',fontsize=14)
plt.legend(fontsize=14)
#plt.savefig("./plots/quantile_plot_2D")
plt.show()

# 1 - $CL_b$ plots

In [ ]:
N_b_pseudo_85 = [] 
N_pseudo_exp = 10000
N_sigPlusbkg_pseudo_85 = []
N_sigPlusbkg_pseudo_90 = []
N_sigPlusbkg_pseudo_95 = []

for i in range(N_pseudo_exp) :
    N_b_pseudo_85.append(np.random.poisson(lam=bkg_histos[0]))
    
    N_sigPlusbkg_pseudo_85.append(np.random.poisson(lam=sig_histos[0]+bkg_histos[0]))
    N_sigPlusbkg_pseudo_90.append(np.random.poisson(lam=sig_histos[1]+bkg_histos[1]))
    N_sigPlusbkg_pseudo_95.append(np.random.poisson(lam=sig_histos[2]+bkg_histos[2]))

    

In [ ]:
llr_Pseudo_obs_85 = stat.LogLikRatioObserved(background=bkg_histos[0],signals=sig_histos,data=N_b_pseudo_85)[0]


llr_Pseudo_obs_sigPlusbkg_85 = stat.LogLikRatioObserved(background=bkg_histos[0],
                                                    signals=sig_histos,
                                                    data=N_sigPlusbkg_pseudo_85)[0]

llr_Pseudo_obs_sigPlusbkg_90 = stat.LogLikRatioObserved(background=bkg_histos[1],
                                                    signals=sig_histos,
                                                    data=N_sigPlusbkg_pseudo_90)[1]

llr_Pseudo_obs_sigPlusbkg_95 = stat.LogLikRatioObserved(background=bkg_histos[2],
                                                    signals=sig_histos,
                                                    data=N_sigPlusbkg_pseudo_95)[2]



In [ ]:
#CL_omb = stat.GetCLOneMinusb(array=llr_85[0],cut=llr_Pseudo_obs_85) 

CL_OneMinusCLb_sigPlusbkg_85 = stat.GetCLOneMinusb(array=llr_85[0],cut=llr_Pseudo_obs_sigPlusbkg_85)
CL_OneMinusCLb_sigPlusbkg_90 = stat.GetCLOneMinusb(array=llr_90[0],cut=llr_Pseudo_obs_sigPlusbkg_90)
CL_OneMinusCLb_sigPlusbkg_95 = stat.GetCLOneMinusb(array=llr_95[0],cut=llr_Pseudo_obs_sigPlusbkg_95)


In [ ]:
mean_85 = np.mean(CL_OneMinusCLb_sigPlusbkg_85)
mean_90 = np.mean(CL_OneMinusCLb_sigPlusbkg_90)
mean_95 = np.mean(CL_OneMinusCLb_sigPlusbkg_95)


In [ ]:
plt.figure(figsize=(9,6))

plt.plot(MHiggs,[mean_85,mean_90,mean_95],'-.',color='brown',label='expected sig+bkg',linewidth=3)
plt.plot(MHiggs,[0.5,0.5,0.5],'--',color='blue',label='expected bkg',linewidth=3)
plt.plot(MHiggs,[CLlist[0][0],CLlist[1][0],CLlist[2][0]],'r-',label='observed',linewidth=3)

plt.hlines(0.16,85,95,color='k',linewidth=1)
plt.text(95,0.16,r'$1 \sigma$',fontsize=14)
plt.hlines(0.023,85,95,color='k',linewidth=1)
plt.text(95,0.023,r'$2 \sigma$',fontsize=14)
plt.hlines(1.35*10**(-3),85,95,color='k',linewidth=1)
plt.text(95,1.35*10**(-3),r'$3 \sigma$',fontsize=14)

plt.semilogy()
plt.xlim(85,95)
plt.legend(fontsize=14,loc='best')
plt.xlabel(r'$m_{H}$ [GeV]',fontsize=14)
plt.ylabel(r'1 - CL${}_\mathrm{b}$',fontsize=14)

#plt.savefig("./plots/OneMinusCLb_plot")


plt.show()